<a href="https://colab.research.google.com/github/Skantastico/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/LS_DSPT3_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
# imports

from __future__ import print_function
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from keras.utils.data_utils import get_file
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import io



In [2]:
# Text to use, complete works of shakespeare

path = get_file('Shakespeare.txt', origin='https://www.gutenberg.org/files/100/100-0.txt')

with io.open(path, encoding='utf-8') as f:
  text = f.read().lower()
  print('corpus length:', len(text))

5783552/5777367 [==============================] - 3s 0us/step
corpus length: 5573152


In [3]:
# number of characters

chars = sorted(list(set(text)))
print('total chars:', len(chars))

total chars: 79


In [4]:
chars

['\t',
 '\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '}',
 'à',
 'â',
 'æ',
 'ç',
 'è',
 'é',
 'ê',
 'î',
 'œ',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '\ufeff']

In [0]:
# make character and indices interchangable and transmutable

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [8]:
# cut text into sequence of maxlen characters

maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])
  print('nb sequences:', len(sentences))

Streaming output truncated to the last 5000 lines.
nb sequences: 1852706
nb sequences: 1852707
nb sequences: 1852708
nb sequences: 1852709
nb sequences: 1852710
nb sequences: 1852711
nb sequences: 1852712
nb sequences: 1852713
nb sequences: 1852714
nb sequences: 1852715
nb sequences: 1852716
nb sequences: 1852717
nb sequences: 1852718
nb sequences: 1852719
nb sequences: 1852720
nb sequences: 1852721
nb sequences: 1852722
nb sequences: 1852723
nb sequences: 1852724
nb sequences: 1852725
nb sequences: 1852726
nb sequences: 1852727
nb sequences: 1852728
nb sequences: 1852729
nb sequences: 1852730
nb sequences: 1852731
nb sequences: 1852732
nb sequences: 1852733
nb sequences: 1852734
nb sequences: 1852735
nb sequences: 1852736
nb sequences: 1852737
nb sequences: 1852738
nb sequences: 1852739
nb sequences: 1852740
nb sequences: 1852741
nb sequences: 1852742
nb sequences: 1852743
nb sequences: 1852744
nb sequences: 1852745
nb sequences: 1852746
nb sequences: 1852747
nb sequences: 1852748
nb 

In [9]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [13]:
# build LSTM model

print('Building model')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Building model


In [0]:
# make samples

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(100):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [21]:
model.fit(x, y,
          batch_size=64,
          epochs=20,
          callbacks=[print_callback])

Epoch 1/20
29019/29027 [============================>.] - ETA: 0s - loss: 1.7023
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " for our crowned heades we have no roofe"
 for our crowned heades we have no rooferit
                                                                                                
----- diversity: 0.5
----- Generating with seed: " for our crowned heades we have no roofe"
 for our crowned heades we have no rooferie
                                                                           your more valia is hi
----- diversity: 1.0
----- Generating with seed: " for our crowned heades we have no roofe"
 for our crowned heades we have no rooferiter marry sworcusp neg’nttets.
  light. as youn sbegly; what prucle to shame.

ebiul.
whose pranco
29027/29027 [==============================] - 135s 5ms/step - loss: 1.7023
Epoch 2/20
29024/29027 [============================>.] - ETA: 0s - loss: 5.4063
----- Generating text afte

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


     heer thene          n  e  ne                                   herno’    thene thei   
----- diversity: 0.5
----- Generating with seed: "caesar, shalt thou have report
    how '"
caesar, shalt thou have report
    how 'aeen     iue or t w  oufaoretj a mal t  he ea hit loe  t  wuwaratreroe  oeo  t i hi
       hro n  n 
----- diversity: 1.0
----- Generating with seed: "caesar, shalt thou have report
    how '"
caesar, shalt thou have report
    how 'dvyt ma  o!t  ow,tla etdo slend  on o ch maiteho.,obhisust  hr at.   n  oe  tn revtineahhf ’ruw ebla
29027/29027 [==============================] - 136s 5ms/step - loss: 5.4061
Epoch 3/20
29016/29027 [============================>.] - ETA: 0s - loss: 3.3119
----- Generating text after Epoch: 2
----- diversity: 0.2
----- Generating with seed: " and a swearing jack,
that thinks with o"
 and a swearing jack,
that thinks with on  ethelen e neun     c              e      t       lo  e   t    h           e   e    tt    e     t 
----- diversity

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN